# SparkFund

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [18]:
rounds = pd.read_csv("Dataset/rounds2.csv",encoding='iso-8859-1 ')
companies = pd.read_csv("Dataset/companies.csv",encoding='iso-8859-1')
companies.head()
rounds.head()

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
3,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0


In [20]:
companies.describe()

,company_permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
count,18334,18334,16941,17488,18333,16531,16085,16240,16240,14112
unique,18334,18272,16916,8208,4,107,251,779,2586,2171
top,/Organization/Artoo,Blink,http://www.askforoffer.com,Biotechnology,operating,USA,CA,SF Bay Area,San Francisco,01-01-2012
freq,1,4,3,1235,14592,10671,3606,2428,911,731


###  How many unique companies are present in rounds2?

In [23]:
rounds['company_permalink']=rounds['company_permalink'].str.lower()
len(rounds['company_permalink'].unique())

66370

### How many unique companies are present in companies?

In [30]:
companies['company_permalink']=companies['company_permalink'].str.lower()
len(companies['company_permalink'].unique())

18334

### In the companies data frame, which column can be used as the unique key for each company? Write the name  of the column.

In [33]:
temp1 = pd.DataFrame(rounds.company_permalink.unique())
temp2 = pd.DataFrame(companies.company_permalink.unique())
temp2.equals(temp1)

False

### Merge the two data frames so that all variables (columns) in the companies frame are added to the rounds2 data frame. Name the merged frame master_frame. How many observations are present in master_frame?

In [38]:
master_frame = pd.merge(rounds, companies, how = 'inner',on = "company_permalink")
len(master_frame.index)

32082

# Funding Type Analysis

In [49]:
master_frame = master_frame.drop(['funding_round_code', 'funding_round_permalink', 'funded_at','company_permalink', 'homepage_url',
                                 'state_code', 'region', 'city', 'founded_at','status'], axis = 1)
master_frame = master_frame[~(master_frame['raised_amount_usd'].isnull() | master_frame['country_code'].isnull() |
                             master_frame['category_list'].isnull())]
master_frame.shape

(13980, 5)

In [50]:
master_frame.funding_round_type.value_counts()

venture    13980
Name: funding_round_type, dtype: int64

In [51]:
master_frame = master_frame[(master_frame['funding_round_type'] == 'venture') 
                            | (master_frame['funding_round_type'] == 'seed')
                            | (master_frame['funding_round_type'] == 'angel')
                            | (master_frame['funding_round_type'] == 'private_equity')]
master_frame.head()

,funding_round_type,raised_amount_usd,name,category_list,country_code
0,venture,1.000000e-05,#fame,Media,IND
4,venture,2.000000e-06,0-6.com,Curated Web,CHN
8,venture,7.194910e-07,Ondine Biomedical Inc.,Biotechnology,CAN
10,venture,2.000000e-05,H2O.ai,Analytics,USA
11,venture,1.700000e-06,H2O.ai,Analytics,USA


In [52]:
master_frame['raised_amount_usd'] = master_frame['raised_amount_usd']/1000000
master_frame.head()

,funding_round_type,raised_amount_usd,name,category_list,country_code
0,venture,1.000000e-11,#fame,Media,IND
4,venture,2.000000e-12,0-6.com,Curated Web,CHN
8,venture,7.194910e-13,Ondine Biomedical Inc.,Biotechnology,CAN
10,venture,2.000000e-11,H2O.ai,Analytics,USA
11,venture,1.700000e-12,H2O.ai,Analytics,USA


In [53]:
round(master_frame.groupby('funding_round_type').raised_amount_usd.mean(), 2)

funding_round_type
venture    0.0
Name: raised_amount_usd, dtype: float64

In [56]:
master_frame = master_frame[master_frame['funding_round_type'] == 'venture'] 

# Country Analysis 

In [58]:
top9 = master_frame.pivot_table(values = 'raised_amount_usd', index = 'country_code', aggfunc = 'sum')
top9 = top9.sort_values(by = 'raised_amount_usd', ascending = False)
top9 = top9.iloc[:9, ]
top9

,raised_amount_usd
country_code,
USA,1.169029e-07
CHN,1.242103e-08
GBR,4.851652e-09
IND,3.291440e-09
CAN,2.767988e-09
DEU,2.238039e-09
FRA,2.135510e-09
ISR,1.962117e-09
NLD,1.599030e-09


In [60]:
master_frame = master_frame[(master_frame['country_code'] == 'USA')
                            | (master_frame['country_code'] == 'GBR')
                            | (master_frame['country_code'] == 'IND')]

# Sector Analysis 

In [67]:
master_frame['category_list'] = master_frame['category_list'].apply(lambda x: x.split('|')[0])
mapping = pd.read_csv('Dataset/mapping.csv')
mapping.category_list = mapping.category_list.replace({'0':'na', '2.na' :'2.0'}, regex=True)
mapping.head()

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,NaN,0,1,0,0,0,0,0,0,0
1,3D,0,0,0,0,0,1,0,0,0
2,3D Printing,0,0,0,0,0,1,0,0,0
3,3D Technology,0,0,0,0,0,1,0,0,0
4,Accounting,0,0,0,0,0,0,0,0,1


In [68]:
mapping = pd.melt(mapping, id_vars =['category_list'], value_vars =['Manufacturing','Automotive & Sports',
                                                              'Cleantech / Semiconductors','Entertainment',
                                                             'Health','News, Search and Messaging','Others',
                                                             'Social, Finance, Analytics, Advertising']) 
mapping = mapping[~(mapping.value == 0)]
mapping = mapping.drop('value', axis = 1)
mapping = mapping.rename(columns = {"variable":"main_sector"})
mapping.head()

,category_list,main_sector
1,3D,Manufacturing
2,3D Printing,Manufacturing
3,3D Technology,Manufacturing
7,Advanced Materials,Manufacturing
15,Agriculture,Manufacturing


In [69]:
master_frame = master_frame.merge(mapping, how = 'left', on ='category_list')
master_frame.head()

,funding_round_type,raised_amount_usd,name,category_list,country_code,main_sector
0,venture,1.000000e-11,#fame,Media,IND,Entertainment
1,venture,2.000000e-11,H2O.ai,Analytics,USA,"Social, Finance, Analytics, Advertising"
2,venture,1.700000e-12,H2O.ai,Analytics,USA,"Social, Finance, Analytics, Advertising"
3,venture,8.900000e-12,H2O.ai,Analytics,USA,"Social, Finance, Analytics, Advertising"
4,venture,5.000000e-12,1 Mainstream,Apps,USA,"News, Search and Messaging"


In [70]:
len(master_frame[master_frame.main_sector.isnull()])

40

In [72]:
master_frame = master_frame[~(master_frame.main_sector.isnull())]
len(master_frame.index)

11298

# Language Analysis 